In [32]:
# Packages
import psycopg2
from psycopg2 import OperationalError
from psycopg2 import sql

import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import networkx as nx

from datetime import datetime, timedelta
from IPython.display import HTML

import os
from dotenv import load_dotenv
dotenv_path = os.path.expanduser('~/.pwds.env')
load_dotenv(dotenv_path)



# Conexão à base de dados
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')


conn = psycopg2.connect(
                host = db_host,
                port = db_port,
                user = db_user,
                password = db_password
        )

In [33]:
cur = conn.cursor()
cur.execute('''ROLLBACK;''')

In [34]:
def h(df):
    """
    Função que permite ver dataframe completa e de forma mais organizada
    """
    return HTML(df.to_html(index=False))

# Análise Contratos 2018

In [35]:
cur = conn.cursor()


cur.execute('''
            SELECT concursos_publicos."nif2",  SUBSTRING(cpv,1,2)
            FROM concursos_publicos
            JOIN contratos_basegov ON contratos_basegov."id" = concursos_publicos."id"
            WHERE EXTRACT(YEAR FROM concursos_publicos."data_publicacao") = %s
            ;
            ''', (2018,))


dados = pd.DataFrame(cur.fetchall())

AmbiguousColumn: column reference "cpv" is ambiguous
LINE 2: ...       SELECT concursos_publicos."nif2",  SUBSTRING(cpv,1,2)
                                                               ^


In [3]:
# Número de contratos
total_contracts = len(dados)

In [4]:
# Nomear colunas
dados = dados.rename(columns = {0:'nif', 1:'cpv'})

In [123]:
# Lista de CPVs
CPV = dados["cpv"].unique()
CPV

array(['45', '90', '77', '60', '33', '79', '15', '66', '71', '80', '39',
       '30', '85', '31', '34', '44', '38', '32', '42', '35', '63', '14',
       '72', '09', '50', '48', '16', '24', '64', '98', '03', '18', '55',
       '65', '22', '37', '92', '43', '75', '51', '19', '70', '73', '76',
       '41'], dtype=object)

**Análise para contratos com CPV começado por 33**

In [6]:
subset = dados[dados["cpv"] == '33']
subset

,nif,cpv
4,503552461,33
9,980571111,33
14,503989037,33
15,501412727,33
16,506708616,33
...,...,...
11426,508592909,33
11430,503989037,33
11495,504594214,33
11533,504594214,33


In [7]:
# NIFs que venceram contratos
unique_nif = subset["nif"].unique()

# Número de NIFs contratados
L = len(unique_nif)
L

356

In [8]:
# NCG : número de concursos ganhos
df = pd.DataFrame(columns = ['NIF', 'NCG','%'])

for i,j in enumerate(unique_nif):
    s = subset[subset["nif"] == j]

    p = round(len(s)/L*100,1)
    if p > 10:
        df.loc[i,:] = j, len(s),  round(len(s)/L*100,1)

In [9]:
df.sort_values(by = "NCG", ascending=False)

,NIF,NCG,%
18,504223933,90,25.3
5,501506543,55,15.4
9,500153370,40,11.2
17,508440246,38,10.7
109,500684324,38,10.7


***
# Generalização
***

In [10]:
for k in CPV:
    subset = dados[dados["cpv"] == k]

    # NIFs que venceram contratos
    unique_nif = subset["nif"].unique()
    
    # Número de contratos por CPV
    L = len(subset)

    df = pd.DataFrame(columns = ['NIF', 'NCG','%'])

    for i,j in enumerate(unique_nif):
        s = subset[subset["nif"] == j]
    
        p = round(len(s)/L*100,1)
        if p > 5:
            df.loc[i,:] = j, len(s),  round(len(s)/L*100,1)

    print('CPV: ', k, '\nNr total concursos: ', len(subset), '\n')
    print(df.sort_values(by = "NCG", ascending=False), '\n\n')
    
    df = pd.DataFrame(columns = ['NIF', 'NCG','%'])

CPV:  45 
Nr total concursos:  2168 

Empty DataFrame
Columns: [NIF, NCG, %]
Index: [] 


CPV:  90 
Nr total concursos:  479 

         NIF NCG    %
7  503210560  31  6.5
0  514073284  26  5.4
6  513966510  26  5.4 


CPV:  77 
Nr total concursos:  193 

         NIF NCG    %
0  504338854  18  9.3
8  503505390  11  5.7 


CPV:  60 
Nr total concursos:  370 

  NIF NCG     %
5   -  74  20.0 


CPV:  33 
Nr total concursos:  1856 

Empty DataFrame
Columns: [NIF, NCG, %]
Index: [] 


CPV:  79 
Nr total concursos:  420 

Empty DataFrame
Columns: [NIF, NCG, %]
Index: [] 


CPV:  15 
Nr total concursos:  679 

         NIF NCG     %
0  503554049  69  10.2
1  500145415  35   5.2 


CPV:  66 
Nr total concursos:  223 

         NIF NCG     %
2  500940231  40  17.9
1  500918880  32  14.3
0  501689168  22   9.9
7  500188629  19   8.5
3  500844321  13   5.8 


CPV:  71 
Nr total concursos:  359 

          NIF NCG    %
10  513246002  28  7.8 


CPV:  80 
Nr total concursos:  88 

          NIF NC

***

In [3]:
today = datetime.now()
yesterday = today - timedelta(days=1)
datapub = yesterday.strftime('%Y-%m-%d')

datapub

'2024-03-18'

In [4]:
cur = conn.cursor()

cur.execute('''
            SELECT contratos_basegov."id"
            FROM contratos_basegov
            WHERE tipo_procedimento = 'Concurso público' AND data_publicacao = %s
            ;
            ''', (datapub,))


ids_onte = cur.fetchall()

In [5]:
(ids_onte[0])

(10616650,)

#### Versão pouco eficiente

In [32]:
def info_contrato(id):
    """
    Função que retorna os primeiros dois dígitos do CPV e NIFs das entidades contrante e contratada
    para apenas um ID

    Argumentos:
    id (tuple): id do contrato. Exemplo: (10614537,)

    returns:
    cpv_xx (str): primeiros dois dígitos do cpv para o respetivo contrato
    nif1 (str): NIF da entidade adjudicante
    nif2 (str): NIF da entidade contratada
    """
    
    cur = conn.cursor()
    cur.execute('''
            SELECT SUBSTRING(cpv,1,2), concursos_publicos."nif1", concursos_publicos."nif2"
            FROM contratos_basegov
            JOIN concursos_publicos ON contratos_basegov."id" = concursos_publicos."id"
            WHERE contratos_basegov."id" = %s
            ;
            ''', (id,))
    
    result = cur.fetchall()[0]

    cpv_xx = result[0]
    nif1 = result[1]
    nif2 = result[2]
    
    return(cpv_xx,nif1,nif2)



def contrats_cpv(cpv):
    """
    Função que retorna o número de concursos públicos celebrados nos últimos 12 meses a partir da data de hoje
    para uma determinada categoria de contratos
    

    Argumentos:
    cpv (str): primeiros dois dígitos do CPV ( categoria de contrato )    
    """
    
    cur = conn.cursor()
    cur.execute('''
                SELECT COUNT(*)
                FROM contratos_basegov
                WHERE tipo_procedimento = 'Concurso público'
                        AND SUBSTRING(contratos_basegov."cpv", 1, 2) = %s
                        AND contratos_basegov.data_publicacao >= CURRENT_DATE - INTERVAL '12 months'
                        AND contratos_basegov.data_publicacao < CURRENT_DATE                
                ;
                ''', (cpv,))
    
    return(cur.fetchone()[0])



def nif2_cpv(cpv,nif2):
    """"
    Função que retorna o número de concursos públicos ganhos por uma determinada entidade adjudicatária 
    para uma determinada categoria de contratos nos últimos 12 meses a partir da data de hoje

    Argumentos:
    cpv: string com os primeiros dois dígitos do CPV ( categoria do contrato )
    nif2: NIF da entidade adjudicatária/vencedora
    """

    cur = conn.cursor()
    cur.execute('''
                SELECT COUNT(*)
                FROM concursos_publicos
                JOIN contratos_basegov ON contratos_basegov."id" = concursos_publicos."id"
                WHERE SUBSTRING(contratos_basegov."cpv", 1, 2) = %s
                          AND concursos_publicos."nif2" = %s
                          AND concursos_publicos.data_publicacao >= CURRENT_DATE - INTERVAL '12 months'
                          AND concursos_publicos.data_publicacao < CURRENT_DATE                
                ;
                ''', (cpv,nif2))
    
    return(cur.fetchone()[0])




def nif1_cpv(cpv,nif1):
    """"
    Função que retorna o número de concursos públicos publicados por uma determinada entidade adjudicante 
    para uma determinada categoria de contratos nos últimos 12 meses a partir da data de hoje

    Argumentos:
    cpv: string com os primeiros dois dígitos do CPV ( categoria do contrato )
    nif1: NIF da entidade adjudicante
    """

    cur = conn.cursor()
    cur.execute('''
                SELECT COUNT(*)
                FROM concursos_publicos
                JOIN contratos_basegov ON contratos_basegov."id" = concursos_publicos."id"
                WHERE SUBSTRING(contratos_basegov."cpv", 1, 2) = %s
                          AND concursos_publicos."nif1" = %s
                          AND concursos_publicos.data_publicacao >= CURRENT_DATE - INTERVAL '12 months'
                          AND concursos_publicos.data_publicacao < CURRENT_DATE
                ;
                ''', (cpv,nif1))
    
    return(cur.fetchone()[0])





def nif12(nif1,nif2,cpv):
    """"
    Função que retorna o número de concursos públicos celebrados entre duas entidadas para uma determinada  
    categoria de contratos nos últimos 12 meses a partir da data de hoje

    Argumentos:
    cpv: string com os primeiros dois dígitos do CPV ( categoria do contrato )
    nif1: NIF da entidade adjudicante
    nif2: NIF da entidade adjudicatária/vencedora
    """

    cur = conn.cursor()
    cur.execute('''
                SELECT COUNT(*)
                FROM concursos_publicos
                JOIN contratos_basegov ON contratos_basegov."id" = concursos_publicos."id"
                WHERE SUBSTRING(contratos_basegov."cpv", 1, 2) = %s
                          AND concursos_publicos."nif1" = %s
                          AND concursos_publicos."nif2" = %s
                          AND concursos_publicos.data_publicacao >= CURRENT_DATE - INTERVAL '12 months'
                          AND concursos_publicos.data_publicacao < CURRENT_DATE
                ;
                ''', (cpv,nif1,nif2))
    
    return(cur.fetchone()[0])

In [7]:
for i in ids_onte[0:2]: 

    # CPV, NIF da entidade adjudicante, NIF da entidade vencedora
    cpv,nif1,nif2 = info_contrato(i)    
    print(cpv,nif1,nif2)
    
    # Número de concursos com primeiros dois dígitos do CPV = cpv
    cpv_12 = contrats_cpv(cpv)
    print(cpv_12)
    
    # Número de concursos ganhos pela entidade vencedora em questão com CPV = cpv
    nif2_won = nif2_cpv(cpv,nif2)
    print(nif2_won)
    
    # Número de concursos lançados pela entidade adjudicante com CPV = cpv
    #nif1_pub = nif1_cpv(cpv,nif1)
    #print(nif1_pub)
    #
    ## Número de concursos celebrados entre entidade adjudicante e vencedora com CPV = cpv
    #nif_last12 = nif12(nif1,nif2,cpv)
    #print(nif_last12)

    print('\n* --- *')

71 506697320 515731765
1235
17

* --- *
33 508080142 506816524
8200
1

* --- *


***

#### Outra abordagem

***

In [35]:
def contract_info(t):
    """
    Função que retorna cpv,nif1 e nif2 para um tuplo de ids de contratos

    Argumentos:
    t (tuple): tuplo de IDs de contratos

    return:
    cpv_xx (list): lista de strings referentes aos primeiros dois do CPV de cada contrato
    nif1 (list): lista de strings referentes ao NIF da entidade adjudicante de cada contrato
    nif2 (list): lista de strings referentes ao NIF da entidade adjudicatária de cada contrato
    """
    
    t = tuple(t)
    
    cur = conn.cursor()
    
    query = '''
            SELECT contratos_basegov."id", SUBSTRING(cpv, 1, 2), concursos_publicos."nif1", concursos_publicos."nif2"
            FROM contratos_basegov
            JOIN concursos_publicos ON contratos_basegov."id" = concursos_publicos."id"
            WHERE contratos_basegov."id" IN %s;
            '''
    cur.execute(query, (t,))
    result = cur.fetchall()

    
    for item in result:

        cpv_xx = list(item[1] for item in result)
        nif1 = list(item[2] for item in result)
        nif2 = list(item[3] for item in result)

    
    return(cpv_xx, nif1, nif2)

cpv_xx, nif1, nif2 = contract_info(ids_onte)

In [38]:
def cpv_contracts():
    """
    Função que retorna o número de contratos celebrados nos últimos 12 meses para cada categoria de CPV

    return:
    result (dataframe): dataframe com duas colunas (categoria do contrato e número de contratos por categoria )
    """
    
    cur = conn.cursor()

    query = '''
            SELECT SUBSTRING(contratos_basegov."cpv", 1, 2) AS cpv_prefix, COUNT(*)
            FROM contratos_basegov
            WHERE tipo_procedimento = 'Concurso público'
                    AND contratos_basegov.data_publicacao >= CURRENT_DATE - INTERVAL '12 months'
                    AND contratos_basegov.data_publicacao < CURRENT_DATE
            GROUP BY cpv_prefix;
            '''
        
    cur.execute(query, (cpv_xx,))
    result = cur.fetchall()

    return pd.DataFrame(result)

In [37]:
cpv_info = cpv_contracts()
cpv_info

,0,1
0,,5
1,03,215
2,09,709
3,14,75
4,15,1432
5,16,58
6,18,297
7,19,83
8,22,117
9,24,223


In [39]:
# Número de contratos com CPV começado por 45 celebrados nos últimos 12 meses
cpv_info.loc[cpv_info.iloc[:, 0] == '45', 1].values[0]

4466

In [44]:
def ramo1(ids, info_cpv):
    """
    Função que compara número de contratos celebrados para uma determinada categoria de contrato ( dada pelos primeiros dois dígitos
    do CPV ) com o número de concursos públicos ganhos por uma determinada entidade adjudicatária para essa mesma categoria

    Argumentos:
    ids (tuple): tuplo de IDs a ser analisado
    info_cpv (dataframe): valores retornados pela função cpv_info
    """
    
    cpv_xx, nif1, nif2 = contract_info(ids)

    for i,j in enumerate(ids):

        # CPV, NIF da entidade adjudicante, NIF da entidade vencedora   
        print(f"CPV: {cpv_xx[i]}\nNIF1: {nif1[i]}\nNIF2: {nif2[i]}")
        
        # Número de concursos com primeiros dois dígitos do CPV = cpv
        cpv_12 = cpv_info.loc[cpv_info.iloc[:, 0] == cpv_xx[i], 1].values[0]
        print(f"Número de concursos celebrados nos últimos 12 meses com CPV começado por {cpv_xx[i]}: {cpv_12}")
        
        # Número de concursos ganhos pela entidade vencedora em questão com CPV = cpv
        nif2_won = nif2_cpv(cpv_xx[i],nif2[i])
        print(f"Número de concursos ganhos nos últimos 12 meses com CPV começado por {cpv_xx[i]} pela entidade adjudicante: {nif2_won}")
    
        print('\n* --- *')

ramo1(ids_onte, cpv_info)

CPV: 33
NIF1: CHULC
NIF2: 500684324
Número de concursos celebrados nos últimos 12 meses com CPV começado por 33: 8224
Número de concursos ganhos nos últimos 12 meses com CPV começado por 33 pela entidade adjudicante: 162

* --- *
CPV: 33
NIF1: 508080142
NIF2: 500042772
Número de concursos celebrados nos últimos 12 meses com CPV começado por 33: 8224
Número de concursos ganhos nos últimos 12 meses com CPV começado por 33 pela entidade adjudicante: 66

* --- *
CPV: 33
NIF1: CHULC
NIF2: 504223933
Número de concursos celebrados nos últimos 12 meses com CPV começado por 33: 8224
Número de concursos ganhos nos últimos 12 meses com CPV começado por 33 pela entidade adjudicante: 297

* --- *
CPV: 33
NIF1: 508080142
NIF2: 501506543
Número de concursos celebrados nos últimos 12 meses com CPV começado por 33: 8224
Número de concursos ganhos nos últimos 12 meses com CPV começado por 33 pela entidade adjudicante: 275

* --- *
CPV: 33
NIF1: 508080142
NIF2: 506470113
Número de concursos celebrados no

In [45]:
def ramo2(ids, info_cpv):
    """
    Função que compara número de contratos celebrados para uma determinada categoria de contrato ( dada pelos primeiros dois dígitos
    do CPV ) com o número de concursos públicos ganhos por uma determinada entidade adjudicatária para essa mesma categoria

    Argumentos:
    ids (tuple): tuplo de IDs a ser analisado
    info_cpv (dataframe): valores retornados pela função cpv_info
    """
    
    cpv_xx, nif1, nif2 = contract_info(ids)

    for i,j in enumerate(ids):
        
        # Número de concursos lançados pela entidade adjudicante com CPV = cpv
        nif1_pub = nif1_cpv(cpv_xx[i],nif1[i])
        print(f"Número de concursos lançados pela entidade adjudicante com CPV: {cpv_xx[i]}: {nif1_pub}")

        
        ## Número de concursos celebrados entre entidade adjudicante e vencedora com CPV = cpv
        nif_last12 = nif12(nif1[i],nif2[i],cpv_xx[i])
        print(f"Número de concursos celebrados entre entidade adjudicante e adjudicatária com CPV: {cpv_xx[i]}: {nif_last12}")

    
        print('\n* --- *')


ramo2(ids_onte, cpv_info)

Número de concursos lançados pela entidade adjudicante com CPV: 33: 505
Número de concursos celebrados entre entidade adjudicante e adjudicatária com CPV: 33: 15

* --- *
Número de concursos lançados pela entidade adjudicante com CPV: 33: 559
Número de concursos celebrados entre entidade adjudicante e adjudicatária com CPV: 33: 10

* --- *
Número de concursos lançados pela entidade adjudicante com CPV: 33: 505
Número de concursos celebrados entre entidade adjudicante e adjudicatária com CPV: 33: 21

* --- *
Número de concursos lançados pela entidade adjudicante com CPV: 33: 559
Número de concursos celebrados entre entidade adjudicante e adjudicatária com CPV: 33: 19

* --- *
Número de concursos lançados pela entidade adjudicante com CPV: 33: 559
Número de concursos celebrados entre entidade adjudicante e adjudicatária com CPV: 33: 3

* --- *
Número de concursos lançados pela entidade adjudicante com CPV: 33: 559
Número de concursos celebrados entre entidade adjudicante e adjudicatária 

# Ramo 1

### Comparar número de contratos celebrados pelo NIF2 com CPV_XXX com o número de contratos celebrados com CPV_XXX

Sempre que é analisado um conjunto de contratos: 

1. Identificar contrato pelo ID
2. Identificar os 3 primeiros dígitos do CPV : categoria
3. Identificar o NIF da entidade vencedora : NIF2
4. Calcular o número de contratos celebrados para a respetiva categoria : X
5. Calcular o número de contratos ganhos pela entidade com NIF2 para a mesma categoria identificada no ponto 2 : Y
6. Calcular o rácio Y/X. Quanto mais próximo de 1, mais suspeito

In [36]:
def contracts(n,cpv):
    """
    Função que retorna os IDs referentes a uma determinada categoria de CPV

    Argumentos:
    n (int): número de dígitos a considerar no slice do CPV ( 2: menor granularidade | 8: muito específico)
    cpv (str): primeiros n dígitos do cpv. Número de dígitos tem de coincidir com n, caso contrário retorna sempre 0 

    return: (tuple) IDs
    """
    
    cur = conn.cursor()

    query = '''
        SELECT concursos_publicos."id", concursos_publicos."data_publicacao", SUBSTRING(concursos_publicos."cpv",1,%s), nif2, adjudicataria, preco_contratual
        FROM concursos_publicos
        WHERE SUBSTRING(cpv,1,%s) = %s;
        '''

    cur.execute(query,(n,n,cpv,))
    result = cur.fetchall()

    return result

In [37]:
gas = contracts(3,'652')
agua = contracts(3,'651')
energia = contracts(3,'653')
travel = contracts(3,'635')
internet = contracts(3,'724')
op_system = contracts(4,'4862')

In [38]:
print(f"Número de contratos para gás: {len(gas)}")
print(f"Número de contratos para água: {len(agua)}")
print(f"Número de contratos para energia: {len(energia)}")
print(f"Número de contratos para viagens: {len(travel)}")
print(f"Número de contratos para internet: {len(internet)}")
print(f"Número de contratos para sistemas operativos: {len(op_system)}")

Número de contratos para gás: 71
Número de contratos para água: 66
Número de contratos para energia: 234
Número de contratos para viagens: 822
Número de contratos para internet: 149
Número de contratos para sistemas operativos: 23


In [39]:
# Empty dataframes
def data_frame(*args):
    
    df = pd.DataFrame(columns=args)
    return df

energia_df =   data_frame('id', 'data', 'cpv', 'nif2', 'entidade', 'nrcontratos_nif2', 'nrcontratos_cpv', '%_contratos', 'preco_contr', 'preco_last_12', '%_preco', 'pr_anocivil', '%_preco2')
gas_df =       data_frame('id', 'data', 'cpv', 'nif2', 'entidade', 'nrcontratos_nif2', 'nrcontratos_cpv', '%_contratos', 'preco_contr', 'preco_last_12', '%_preco', 'pr_anocivil', '%_preco2')
agua_df =      data_frame('id', 'data', 'cpv', 'nif2', 'entidade', 'nrcontratos_nif2', 'nrcontratos_cpv', '%_contratos', 'preco_contr', 'preco_last_12', '%_preco', 'pr_anocivil', '%_preco2')
travel_df =    data_frame('id', 'data', 'cpv', 'nif2', 'entidade', 'nrcontratos_nif2', 'nrcontratos_cpv', '%_contratos', 'preco_contr', 'preco_last_12', '%_preco', 'pr_anocivil', '%_preco2')
internet_df =  data_frame('id', 'data', 'cpv', 'nif2', 'entidade', 'nrcontratos_nif2', 'nrcontratos_cpv', '%_contratos', 'preco_contr', 'preco_last_12', '%_preco', 'pr_anocivil', '%_preco2')
op_system_df = data_frame('id', 'data', 'cpv', 'nif2', 'entidade', 'nrcontratos_nif2', 'nrcontratos_cpv', '%_contratos', 'preco_contr', 'preco_last_12', '%_preco', 'pr_anocivil', '%_preco2')

In [45]:
contrato_teste = gas[0]
labels = ['ID', 'Data', 'CPV', 'NIF', 'Entidade', 'PreçoContratual'
         ]
for i,j in enumerate(contrato_teste):
    print(labels[i],':',j)

ID : 9691451
Data : 2023-01-10
CPV : 652
NIF : 500697370
Entidade : Petrogal, S.A.
PreçoContratual : 792000.0


In [57]:
id_t, data_t, cpv_t, nif2_t, entidade_t, precocontr_t = contrato_teste

In [9]:
def cpv3_contracts():
    """
    Função que retorna o número de contratos celebrados nos últimos 12 meses para cada categoria de 3 dígitos do CPV

    return:
    result (dataframe): dataframe com duas colunas (categoria do contrato e número de contratos por categoria )
    """
    
    cur = conn.cursor()

    query = '''
            SELECT SUBSTRING(contratos_basegov."cpv", 1, 3) AS cpv_prefix, COUNT(*)
            FROM contratos_basegov
            WHERE tipo_procedimento = 'Concurso público'
                    AND contratos_basegov.data_publicacao >= CURRENT_DATE - INTERVAL '12 months'
                    AND contratos_basegov.data_publicacao <= CURRENT_DATE
            GROUP BY cpv_prefix;
            '''
        
    cur.execute(query,)
    result = cur.fetchall()

    return pd.DataFrame(result)

In [10]:
def info_nif2(id):
    """
    Função que retorna NIF da entidade vencedora para um determinado contrato

    Argumento:
    id (tuple): ID do contrato em questão

    return: (tuple) NIF
    """
    
    cur = conn.cursor()
    cur.execute('''
            SELECT concursos_publicos."nif2"
            FROM concursos_publicos
            WHERE concursos_publicos."id" = %s
            ;
            ''', (id,))
    
    result = cur.fetchall()[0]
    
    return(result)

***
- Calcular número de contratos celebrados pela entidade adjudicatária - NIF2 - para o CPV em questão
- Número total de contratos celebrados para o CPV em questão

In [116]:
def contracts_number_nif2(dia,cpv,n,nif,yearly):
    """
    Função que retorna o número de contratos ganhos por uma determinada entidade para uma categoria de contratos

    Argumentos:
    dia (date YYYY-MM-DD): data em que o contrato foi publicado no portal Base
    cpv (str): primeiros n dígitos do CPV 
    n (int): número de dígitos a considerar na categorização do CPV
    yearly (str): Se for 'True' calcula apenas para os últimos 12 meses a partir de um determinado dia
                  Se for 'False' calcula para a totalidade de contratos presente na base de dados

    return (int): número de contratos celebrados 
    """

    if yearly == 'True':
        cur = conn.cursor()
        cur.execute('''
                    SELECT COUNT(*)
                    FROM concursos_publicos
                    WHERE SUBSTRING(concursos_publicos."cpv", 1, %s) = %s
                              AND concursos_publicos."nif2" = %s               
                    ;
                    ''', (n,cpv,nif))
        return(cur.fetchone()[0])
        
    elif yearly == 'False':
        cur = conn.cursor()
        cur.execute('''
                    SELECT COUNT(*)
                    FROM concursos_publicos
                    WHERE SUBSTRING(concursos_publicos."cpv", 1, %s) = %s
                              AND concursos_publicos."nif2" = %s
                              AND concursos_publicos.data_publicacao >= %s - INTERVAL '12 months'
                              AND concursos_publicos.data_publicacao <= %s                
                    ;
                    ''', (n,cpv,nif,dia,dia))
        return(cur.fetchone()[0])

    
    elif yearly == 'Civil':
        ano = dia.year
        cur = conn.cursor()
        cur.execute('''
                    SELECT COUNT(*)
                    FROM concursos_publicos
                    WHERE SUBSTRING(concursos_publicos."cpv", 1, %s) = %s
                            AND concursos_publicos."nif2" = %s
                            AND EXTRACT(YEAR FROM concursos_publicos.data_publicacao) = %s
                    ;
                    ''', (n,cpv,nif,ano))
        return(cur.fetchone()[0])

    
    else:
        return('Error: yearly input not valid')

In [115]:
def contracts_number(dia,cpv,n,yearly):
    """
    Função que retorna o número de concursos públicos celebrados para uma determinada categoria de contratos
    
    Argumentos:
    dia (date YYYY-MM-DD): data em que o contrato foi publicado no portal Base
    cpv (str): primeiros n dígitos do CPV 
    n (int): número de dígitos a considerar na categorização do CPV
    yearly (str): Se for 'True' calcula apenas para os últimos 12 meses a partir de um determinado dia
                  Se for 'False' calcula para a totalidade de contratos presente na base de dados


    return (int): número de concursos celebrados 
    """
    
    if yearly == 'True':
        cur = conn.cursor()
        cur.execute('''
                    SELECT COUNT(*)
                    FROM concursos_publicos
                    WHERE SUBSTRING(concursos_publicos."cpv", 1, %s) = %s               
                    ;
                    ''', (n,cpv))
        return(cur.fetchone()[0])

    
    elif yearly == 'False':
        cur = conn.cursor()
        cur.execute('''
                    SELECT COUNT(*)
                    FROM concursos_publicos
                    WHERE SUBSTRING(concursos_publicos."cpv", 1, %s) = %s
                            AND concursos_publicos.data_publicacao >= %s - INTERVAL '12 months'
                            AND concursos_publicos.data_publicacao <= %s                
                    ;
                    ''', (n,cpv,dia,dia))
        return(cur.fetchone()[0])


    elif yearly == 'Civil':
        ano = dia.year
        cur = conn.cursor()
        cur.execute('''
                    SELECT COUNT(*)
                    FROM concursos_publicos
                    WHERE SUBSTRING(concursos_publicos."cpv", 1, %s) = %s
                            AND EXTRACT(YEAR FROM concursos_publicos.data_publicacao) = %s
                    ;
                    ''', (n,cpv,ano))
        return(cur.fetchone()[0])


    else:
        return('Error: yearly input not valid')

In [86]:
x1 = contracts_number_nif2(data_t,cpv_t,len(cpv_t),nif2_t,'True')
x2 = contracts_number(data_t,cpv_t,len(cpv_t),'True')

In [87]:
print(f"Entidade: {entidade_t} \nCPV: {cpv_t} \
        \nNúmero total de concursos ganhos por {entidade_t} para CPV {cpv_t}: {x1} \
        \nNúmero total de concursos celebrados para CPV {cpv_t}: {x2} \
        \nFração de concursos ganhos por {entidade_t}: {round(x1/x2*100,1)}%")

Entidade: Petrogal, S.A. 
CPV: 652         
Número total de concursos ganhos por Petrogal, S.A. para CPV 652: 17         
Número total de concursos celebrados para CPV 652: 71         
Fração de concursos ganhos por Petrogal, S.A.: 23.9%


***
- Calcular número de contratos celebrados, nos últimos 12 meses, pela entidade adjudicatária - NIF2 - para o CPV em questão
- Número total de contratos celebrados nos últimos 12 meses para o CPV em questão

In [106]:
x3 = contracts_number_nif2(data_t,cpv_t,len(cpv_t),nif2_t,'False')
x4 = contracts_number(data_t,cpv_t,len(cpv_t),'False')

In [82]:
print(f"Entidade: {entidade_t} \nCPV: {cpv_t} \
        \nNúmero total de concursos ganhos por {entidade_t} para CPV {cpv_t} nos últimos 12 meses: {x3} \
        \nNúmero total de concursos celebrados para CPV {cpv_t} nos últimos 12 meses: {x4} \
        \nFração de concursos ganhos por {entidade_t}: {round(x3/x4*100,1)}%")

Entidade: Petrogal, S.A. 
CPV: 652         
Número total de concursos ganhos por Petrogal, S.A. para CPV 652 nos últimos 12 meses: 5         
Número total de concursos celebrados para CPV 652 nos últimos 12 meses: 7         
Fração de concursos ganhos por Petrogal, S.A.: 71.4%


***
- Calcular número de contratos celebrados, no presente ano civil, pela entidade adjudicatária - NIF2 - para o CPV em questão
- Número total de contratos celebrados, no presente ano civil, para o CPV em questão

In [117]:
x5 = contracts_number_nif2(data_t,cpv_t,len(cpv_t),nif2_t,'Civil')
x6 = contracts_number(data_t,cpv_t,len(cpv_t),'Civil')

In [118]:
print(f"Entidade: {entidade_t} \nCPV: {cpv_t} \
        \nNúmero total de concursos ganhos por {entidade_t} para CPV {cpv_t} no ano {data_t.year}: {x5} \
        \nNúmero total de concursos celebrados para CPV {cpv_t} no ano {data_t.year}: {x6} \
        \nFração de concursos ganhos por {entidade_t}: {round(x5/x6*100,1)}%")

Entidade: Petrogal, S.A. 
CPV: 652         
Número total de concursos ganhos por Petrogal, S.A. para CPV 652 no ano 2023: 6         
Número total de concursos celebrados para CPV 652 no ano 2023: 13         
Fração de concursos ganhos por Petrogal, S.A.: 46.2%


<div class="verticalgap" style="height:10cm"> </div>

In [119]:
def contracts_price(dia,cpv,n,yearly):
    """
    Função que retorna o preço contratual total de concursos públicos celebrados para uma determinada categoria de contratos
    
    Argumentos:
    dia (date YYYY-MM-DD): data em que o contrato foi publicado no portal Base
    cpv (str): primeiros n dígitos do CPV 
    n (int): número de dígitos a considerar na categorização do CPV
    yearly (str): Se for 'True' calcula apenas para os últimos 12 meses a partir de um determinado dia
                  Se for 'False' calcula para a totalidade de contratos presente na base de dados


    return (int): número de concursos celebrados 
    """

    if yearly == 'True':
        cur = conn.cursor()
        cur.execute('''
                    SELECT SUM(preco_contratual)
                    FROM concursos_publicos
                    WHERE SUBSTRING(concursos_publicos."cpv", 1, %s) = %s               
                    ;
                    ''', (n,cpv))
        return(cur.fetchone()[0])


    elif yearly == 'False':
        cur = conn.cursor()
        cur.execute('''
                    SELECT SUM(preco_contratual)
                    FROM concursos_publicos
                    WHERE SUBSTRING(concursos_publicos."cpv", 1, %s) = %s
                            AND concursos_publicos.data_publicacao >= %s - INTERVAL '12 months'
                            AND concursos_publicos.data_publicacao <= %s                
                    ;
                    ''', (n,cpv,dia,dia))
        return(cur.fetchone()[0])

    
    elif yearly == 'Civil':
        ano = dia.year
        cur = conn.cursor()
        cur.execute('''
                    SELECT SUM(preco_contratual)
                    FROM concursos_publicos
                    WHERE SUBSTRING(concursos_publicos."cpv", 1, %s) = %s
                            AND EXTRACT(YEAR FROM concursos_publicos.data_publicacao) = %s               
                    ;
                    ''', (n,cpv,ano))
        return(cur.fetchone()[0])

    
    else:
        return('Error: yearly input not valid')

***

In [128]:
y1 = contracts_price(data_t,cpv_t,len(cpv_t),'True')

In [130]:
print(f"Entidade: {entidade_t} \nCPV: {cpv_t} \
        \nValor adjudicado para o concurso ganho por {entidade_t} para CPV {cpv_t}: {round(precocontr_t,2)}€ \
        \nValor total adjudicado para concursos com CPV {cpv_t}: {round(y1,2)}€ \
        \nFração de concursos ganhos por {entidade_t}: {round(precocontr_t/y1*100,1)}%")

Entidade: Petrogal, S.A. 
CPV: 652         
Valor adjudicado para o concurso ganho por Petrogal, S.A. para CPV 652: 792000.0€         
Valor total adjudicado para concursos com CPV 652: 17449122.37€         
Fração de concursos ganhos por Petrogal, S.A.: 4.5%


***

In [131]:
y2 = contracts_price(data_t,cpv_t,len(cpv_t),'False')

In [132]:
print(f"Entidade: {entidade_t} \nCPV: {cpv_t} \
        \nValor adjudicado para o concurso ganho por {entidade_t} para CPV {cpv_t}: {round(precocontr_t,2)}€ \
        \nValor total adjudicado para concursos com CPV {cpv_t} nos últimos 12 meses: {round(y2,2)}€ \
        \nFração de concursos ganhos por {entidade_t}: {round(precocontr_t/y2*100,1)}%")

Entidade: Petrogal, S.A. 
CPV: 652         
Valor adjudicado para o concurso ganho por Petrogal, S.A. para CPV 652: 792000.0€         
Valor total adjudicado para concursos com CPV 652 nos últimos 12 meses: 2360783.41€         
Fração de concursos ganhos por Petrogal, S.A.: 33.5%


***
- Calcular número de contratos celebrados, no presente ano civil, pela entidade adjudicatária - NIF2 - para o CPV em questão
- Número total de contratos celebrados, no presente ano civil, para o CPV em questão

In [133]:
y3 = contracts_price(data_t,cpv_t,len(cpv_t),'Civil')

In [134]:
print(f"Entidade: {entidade_t} \nCPV: {cpv_t} \
        \nValor adjudicado para o concurso ganho por {entidade_t} para CPV {cpv_t}: {round(precocontr_t,2)}€ \
        \nValor total adjudicado para concursos com CPV {cpv_t} nos últimos 12 meses: {round(y3,2)}€ \
        \nFração de concursos ganhos por {entidade_t}: {round(precocontr_t/y3*100,1)}%")

Entidade: Petrogal, S.A. 
CPV: 652         
Valor adjudicado para o concurso ganho por Petrogal, S.A. para CPV 652: 792000.0€         
Valor total adjudicado para concursos com CPV 652 nos últimos 12 meses: 5096350.76€         
Fração de concursos ganhos por Petrogal, S.A.: 15.5%


<div class="verticalgap" style="height:10cm"> </div>

In [164]:
gasgas = pd.DataFrame(gas)
gas_unique_nifs = gasgas.iloc[:,3].unique()

gasgas.head(1)

,0,1,2,3,4,5
0,9691451,2023-01-10,652,500697370,"Petrogal, S.A.",792000.0


In [182]:
df_gas_teste = pd.DataFrame(columns=['Entidade', 'NIF', 'PreçoTotal Adjudicado_NIF2','PreçoTotal Adjudicado_CPV', 'Rácio', 'NrConcGanhos', 'NrConcTotal', 'Rácio'])

for i,j in enumerate(gas_unique_nifs):
    
    entity = gasgas[gasgas[gasgas.columns[3]] == j].iloc[0, 4]
    
    total_price_nif = round(sum(gasgas[gasgas[gasgas.columns[3]] == j].iloc[:, 5]),2)
    total_price_cpv = round(contracts_price(data_t,cpv_t,len(cpv_t),'True'),2)
    racio1 = round(total_price_nif/total_price_cpv*100,2)

    concursos_nif = len(gasgas[gasgas[gasgas.columns[3]] == j].iloc[:, 5])
    n = len(gasgas)
    racio2 = round(concursos_nif/n*100,2)
    
    df_gas_teste.loc[i] = entity,j,total_price_nif, total_price_cpv, racio1, concursos_nif, n, racio2

In [183]:
df_gas_teste

,Entidade,NIF,PreçoTotal Adjudicado_NIF2,PreçoTotal Adjudicado_CPV,Rácio,NrConcGanhos,NrConcTotal,Rácio
0,"Petrogal, S.A.",500697370,4107357.24,17449122.37,23.54,17,71,23.94
1,"Goldenergy - Comercializadora de Energia, S. A.",507857542,2223057.25,17449122.37,12.74,6,71,8.45
2,"Galp Power, SA",504723456,2041191.79,17449122.37,11.70,16,71,22.54
3,edp comercial,503504564,4007335.25,17449122.37,22.97,7,71,9.86
4,Valorsul - Valorização e Tratamento de Resíduo...,509479600,200000.00,17449122.37,1.15,1,71,1.41
5,"OZ Energia Gás, S.A.",500099790,484538.58,17449122.37,2.78,3,71,4.23
6,GÁS NATURAL COMERCIALIZADORA S.A. – SUCURSAL E...,980393388,2867824.88,17449122.37,16.44,8,71,11.27
7,"SOCIEDADE PORTUGUESA DO ""ARLÍQUIDO"", LDA",500261512,12121.72,17449122.37,0.07,1,71,1.41
8,"Repsol Gás Portugal, Sa.",507039440,897467.50,17449122.37,5.14,6,71,8.45
9,Dourogás Natural - Medição e Explopração de Si...,509828698,303771.38,17449122.37,1.74,1,71,1.41


<div class="verticalgap" style="height:10cm"> </div>

In [198]:
def info_contrato(id):
    """
    Função que retorna CPV da entidade vencedora para um determinado contrato

    Argumento:
    id (tuple): ID do contrato em questão

    return: (str) CPV
    """
    
    cur = conn.cursor()
    cur.execute('''
            SELECT concursos_publicos."data_publicacao", SUBSTRING(concursos_publicos."cpv",1,3), nif2, adjudicataria, preco_contratual
            FROM concursos_publicos
            WHERE concursos_publicos."id" = %s
            ;
            ''', (id,))
    
    result = cur.fetchall()[0]
    
    return(result)

In [212]:
def r025(id):
    """
    """
    
    data_t,cpv_t,nif2_t,entidade_t,precocontr_t = info_contrato(id)

    x1 = contracts_number_nif2(data_t,cpv_t,len(cpv_t),nif2_t,'True')
    x2 = contracts_number(data_t,cpv_t,len(cpv_t),'True')
    print(f"Entidade: {entidade_t} \nCPV: {cpv_t} \
            \nNúmero total de concursos ganhos por {entidade_t} para CPV {cpv_t}: {x1} \
            \nNúmero total de concursos celebrados para CPV {cpv_t}: {x2} \
            \nFração de concursos ganhos por {entidade_t}: {round(x1/x2*100,1)}% \n")
    
    
    x3 = contracts_number_nif2(data_t,cpv_t,len(cpv_t),nif2_t,'False')
    x4 = contracts_number(data_t,cpv_t,len(cpv_t),'False')
    print(f"\nNúmero total de concursos ganhos por {entidade_t} para CPV {cpv_t} nos últimos 12 meses: {x3} \
            \nNúmero total de concursos celebrados para CPV {cpv_t} nos últimos 12 meses: {x4} \
            \nFração de concursos ganhos por {entidade_t}: {round(x3/x4*100,1)}% \n")
    
    
    
    x5 = contracts_number_nif2(data_t,cpv_t,len(cpv_t),nif2_t,'Civil')
    x6 = contracts_number(data_t,cpv_t,len(cpv_t),'Civil')
    print(f"\nNúmero total de concursos ganhos por {entidade_t} para CPV {cpv_t} no ano {data_t.year}: {x5} \
            \nNúmero total de concursos celebrados para CPV {cpv_t} no ano {data_t.year}: {x6} \
            \nFração de concursos ganhos por {entidade_t}: {round(x5/x6*100,1)}% \n")
    
    
    
    y1 = contracts_price(data_t,cpv_t,len(cpv_t),'True')
    print(f"\nValor adjudicado para o concurso ganho por {entidade_t} para CPV {cpv_t}: {round(precocontr_t,2)}€ \
            \nValor total adjudicado para concursos com CPV {cpv_t}: {round(y1,2)}€ \
            \nFração de concursos ganhos por {entidade_t}: {round(precocontr_t/y1*100,1)}% \n")
    
    
    y2 = contracts_price(data_t,cpv_t,len(cpv_t),'False')
    print(f"\nValor adjudicado para o concurso ganho por {entidade_t} para CPV {cpv_t}: {round(precocontr_t,2)}€ \
            \nValor total adjudicado para concursos com CPV {cpv_t} nos últimos 12 meses: {round(y2,2)}€ \
            \nFração de concursos ganhos por {entidade_t}: {round(precocontr_t/y2*100,1)}% \n")
    
    
    y3 = contracts_price(data_t,cpv_t,len(cpv_t),'Civil')
    print(f"\nValor adjudicado para o concurso ganho por {entidade_t} para CPV {cpv_t}: {round(precocontr_t,2)}€ \
            \nValor total adjudicado para concursos com CPV {cpv_t} no ano {data_t.year}: {round(y3,2)}€ \
            \nFração de concursos ganhos por {entidade_t}: {round(precocontr_t/y3*100,1)}% \n")
        
    #return nif

In [213]:
r025(gas[0][0])

Entidade: Petrogal, S.A. 
CPV: 652             
Número total de concursos ganhos por Petrogal, S.A. para CPV 652: 17             
Número total de concursos celebrados para CPV 652: 71             
Fração de concursos ganhos por Petrogal, S.A.: 23.9% 


Número total de concursos ganhos por Petrogal, S.A. para CPV 652 nos últimos 12 meses: 5             
Número total de concursos celebrados para CPV 652 nos últimos 12 meses: 7             
Fração de concursos ganhos por Petrogal, S.A.: 71.4% 


Número total de concursos ganhos por Petrogal, S.A. para CPV 652 no ano 2023: 6             
Número total de concursos celebrados para CPV 652 no ano 2023: 13             
Fração de concursos ganhos por Petrogal, S.A.: 46.2% 


Valor adjudicado para o concurso ganho por Petrogal, S.A. para CPV 652: 792000.0€             
Valor total adjudicado para concursos com CPV 652: 17449122.37€             
Fração de concursos ganhos por Petrogal, S.A.: 4.5% 


Valor adjudicado para o concurso ganho por Petr

***

<div class="verticalgap" style="height:10cm"> </div>

***

In [28]:
#contracts_number(dia,cpv,n)
#contracts_number_nif2(dia,cpv,n,nif)


# x1         x2             x3                 x4         x5            x6      
# id  data_publicacao  SUBSTRING(cpv,1,%s)   nif2   adjudicataria     preco_contr


# data_frame('id', 'data', 'cpv', 'nif2', 'entidade', 'nrcontratos_nif2', 'nrcontratos_cpv'
# '%_contratos', 'preco_contr', 'preco_last_12', '%_preco', 'pr_anocivil', '%_preco2')


for i,j in enumerate(gas):

    x1,x2,x3,x4,x5,x6 = j
    
    # Número de dígitos considerado para categorização do CPV
    n = len(x3)

    # Ano civil do contrato
    ano = x2.year

    # Número de contratos celebrados com CPV = x3 para os últimos 12 meses ( desde o dia x2 )
    nr_cpv = contracts_number(x2,x3,n)

    # Número de contratos celebrados pela entidade vencedora nos últimos 12 meses
    nr_nif2 = contracts_number_nif2(x2,x3,n,x4)

    # Montante total, em €, utilizado nesta categoria de CPV nos últimos 12 meses
    price_year = contracts_price(x2,x3,n)

    # Montante total, em €, alocado nesta categoria de CPV para o ano civil
    price_civil = contracts_price_civil(ano,x3,n)
    
    # Fração de dinheiro atribuída à entidade vencedora nos últimos 12 meses
    r1 = round(x6/price_year * 100,1)

    # Fração de contratos ganhos pela entidade vencedora nos últimos 12 meses
    r2 = round(nr_nif2/nr_cpv * 100,1)

    # Fração de dinheiro atribuída à entidade vencedora para o ano civil
    r3 = round(x6/price_civil * 100,1)
    
    gas_df.loc[i] = (x1,x2,x3,x4,x5,nr_nif2,nr_cpv,r2,x6,price_year,r1,price_civil,r3)

In [29]:
nif2_counts = gas_df['nif2'].value_counts()

nif2_counts = gas_df['nif2'].value_counts()
sorted_df = gas_df[gas_df['nif2'].isin(nif2_counts.index)].sort_values(by=['nif2', 'data'], ascending=[False, True])

h(sorted_df)

id,data,cpv,nif2,entidade,nrcontratos_nif2,nrcontratos_cpv,%_contratos,preco_contr,preco_last_12,%_preco,pr_anocivil,%_preco2
4268633,2018-02-20,652,980393388,"Gas Natural Comercializadora, S.A.",1,2,50.0,2.500000e+05,5.500000e+05,45.5,1.398680e+06,17.9
4328068,2018-03-07,652,980393388,"Gás Natural Comercializadora, S.A.",2,3,66.7,6.738013e+04,6.173801e+05,10.9,1.398680e+06,4.8
5811325,2019-09-02,652,980393388,"Gás Natural Comercializadora, S.A.",1,13,7.7,1.625208e+05,4.511419e+06,3.6,4.575959e+06,3.6
7379983,2021-01-11,652,980393388,"Gas Natural Comercializadora, S.A. - Sucursal em Portugal",1,13,7.7,1.210451e+06,2.353954e+06,51.4,2.743180e+06,44.1
7572531,2021-03-18,652,980393388,"Gas Natural Comercializadora, S.A. - Sucursal em Portugal",2,18,11.1,2.666311e+05,3.398934e+06,7.8,2.743180e+06,9.7
9287407,2022-05-30,652,980393388,GÁS NATURAL COMERCIALIZADORA S.A. – SUCURSAL EM PORTUGAL,1,6,16.7,7.543868e+04,5.568768e+05,13.5,1.568783e+06,4.8
9902689,2023-04-01,652,980393388,"Gas Natural Comercializadora, S.A. - Sucursal em Portugal",2,10,20.0,2.572579e+05,4.532660e+06,5.7,5.096351e+06,5.0
10328792,2023-11-07,652,980393388,"Gás Natural Comercializadora, S.A. – Sucursal Em Portugal",2,13,15.4,5.781452e+05,6.144300e+06,9.4,5.096351e+06,11.3
6643404,2020-06-01,652,513239731,"PH ENERGIA, LDA",1,9,11.1,9.617691e+04,2.431055e+06,4.0,1.143503e+06,8.4
6809187,2020-07-29,652,513239731,"PH ENERGIA, LDA",2,11,18.2,8.969147e+04,2.285839e+06,3.9,1.143503e+06,7.8


In [117]:
# Função que generaliza passos das células anteriores 

def geral_df(ids,dframe):
    """
    Argumentos:
    ids (tuple): conjunto de IDs a ser analisado e colunas consideradas necessárias
    dframe (dataframe): dataframe vazia com as seguintes colunas: id,data,nif2,nif2_nr,total_cpv,%,entidade

    return (dataframe): dataframe preenchida para novo conjunto de IDs
    """
    
    for i,j in enumerate(ids):

        # x1 = id | x2 = data_publicacao | x3 = SUBSTRING(cpv,1,%s) | x4 = nif1 | x5 = nif2 | x6 = adjudicataria
        # O input ids é obtido através da função contracts(n,cpv) e a função retorna todos os parâmetros de x1 a x6
        x1,x2,x3,x4,x5,x6 = j
        
        # Número de dígitos considerado para categorização do CPV
        n = len(x3)
    
        # Número de contratos celebrados com CPV = x3 para os últimos 12 meses ( desde o dia x2 )
        nr_cpv = contracts_number(x2,x3,n)
        
        #print(f"Número de contratos celebrados com CPV {x3} nos últimos 12 meses desde o dia {x2}: \t{nr_cpv}")

        # Número de contratos ganhos pela entidade com NIF2 e com CPV = x3 para os últimos 12 meses ( desde o dia x2 )
        nr_nif2 = contracts_number_nif2(x2,x3,n,x5)

        # Rácio
        r = round(nr_nif2/nr_cpv * 100,1)
    
        dframe.loc[i] = (x1,x2,x5,nr_nif2,nr_cpv,r,x6)

    nif2_counts = dframe['nif2'].value_counts()
    sorted_df = dframe[dframe['nif2'].isin(nif2_counts.index)].sort_values(by=['nif2', 'data'], ascending=[False, True])
    
    return(h(sorted_df))


### Gás

In [118]:
geral_df(gas,gas_df)

id,data,nif2,nif2_nr,total_cpv,%,entidade
4268633,2018-02-20,980393388,1,2,50.0,"Gas Natural Comercializadora, S.A."
4328068,2018-03-07,980393388,2,3,66.7,"Gás Natural Comercializadora, S.A."
5811325,2019-09-02,980393388,1,13,7.7,"Gás Natural Comercializadora, S.A."
7379983,2021-01-11,980393388,1,13,7.7,"Gas Natural Comercializadora, S.A. - Sucursal em Portugal"
7572531,2021-03-18,980393388,2,18,11.1,"Gas Natural Comercializadora, S.A. - Sucursal em Portugal"
9287407,2022-05-30,980393388,1,6,16.7,GÁS NATURAL COMERCIALIZADORA S.A. – SUCURSAL EM PORTUGAL
9902689,2023-04-01,980393388,2,10,20.0,"Gas Natural Comercializadora, S.A. - Sucursal em Portugal"
10328792,2023-11-07,980393388,2,13,15.4,"Gás Natural Comercializadora, S.A. – Sucursal Em Portugal"
6643404,2020-06-01,513239731,1,9,11.1,"PH ENERGIA, LDA"
6809187,2020-07-29,513239731,2,11,18.2,"PH ENERGIA, LDA"


### Água

In [141]:
geral_df(agua,agua_df)

id,data,nif2,nif2_nr,total_cpv,%,entidade
10433224,2024-01-05,516559265,1,25,4.0,"Engimaqua - Engenharia e Serviços, Ld.ª"
7655538,2021-05-12,515994740,1,8,12.5,GAZELLE POWER - UNIPESSOAL LDA
10238077,2023-09-13,513339701,1,18,5.6,"Planeta Lógico- Transportes, Unipessoal, Lda"
9992631,2023-05-17,510903010,1,11,9.1,"Enviman – Manutenção de Sistemas Ambientais, Lda."
5999059,2019-11-04,510279201,1,7,14.3,"Sociedade de Construções Silveira da Rocha, S.A."
6098996,2019-12-10,509944647,1,9,11.1,"CONSTRUBUILD, LDA"
8803885,2022-01-27,509944647,1,6,16.7,"CONSTRUBUILD - SERVICES, LIMITADA"
9856240,2023-03-10,509944647,1,7,14.3,"Construbuild - Services, Limitada"
9953192,2023-04-28,509944647,3,9,33.3,"Construbuild - Services, Limitada"
9953123,2023-04-28,509944647,3,9,33.3,"Construbuild - Services, Limitada"


### Energia

In [142]:
geral_df(energia,energia_df)

id,data,nif2,nif2_nr,total_cpv,%,entidade
10460213,2024-01-13,980582393,2,36,5.6,Acciona Green Energy Developments S.L. Sucursal Em Portugal
10460179,2024-01-13,980582393,2,36,5.6,Acciona Green Energy Developments S.L. Sucursal Em Portugal
6703820,2020-06-22,980393388,1,35,2.9,"Gás Natural Comercializadora, S.A - Sucursal em Portugal"
4679151,2018-07-23,980245974,1,20,5.0,"Endesa Energia, S.A. – Sucursal Portugal"
5079372,2018-12-28,980245974,2,36,5.6,"Endesa Energia, S.A. - Sucursal Portugal"
5178941,2019-01-24,980245974,3,34,8.8,"Endesa Energia, S.A. - Sucursal Portugal"
5202895,2019-01-31,980245974,4,37,10.8,"Endesa Energia, S.A. - Sucursal Portugal"
5808001,2019-08-30,980245974,4,44,9.1,"Endesa Energia, S.A. - Sucursal Portugal"
6437996,2020-03-23,980245974,2,36,5.6,"ENDESA ENERGIA, SA - SUCURSAL PORTUGAL"
6496293,2020-04-17,980245974,3,36,8.3,"Endesa Energia, S.A. - Sucursal Portugal"


### Viagens 

In [144]:
#geral_df(travel,travel_df)

In [145]:
net = geral_df(internet,internet_df)

KeyboardInterrupt: 

In [ ]:
opsystem = geral_df(op_system,op_system_df)

In [1]:
255+1271+38

1564

In [3]:
1271/1564

0.8126598465473146